In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pyplot as pyplot
import gc
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_squared_log_error
import lightgbm as lgb
import math
from tqdm import tqdm_notebook as tqdm
import warnings
import datetime as dt
warnings.simplefilter(action='ignore', category=FutureWarning)

PATH = 'data/'
#!ls ../input/ashrae-energy-prediction


**Reduce Memory function**

In [2]:
#Based on this great kernel https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65
def reduce_mem_usage(df, verbose=False):
    start_mem_usg = df.memory_usage().sum() / 1024**2 
    if verbose:
        print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in df.columns:
        if df[col].dtype != object:  # Exclude strings      
            if verbose:
                # Print current column type
                print("******************************")
                print("Column: ",col)
                print("dtype before: ",df[col].dtype)            
            # make variables for Int, max and min
            IsInt = False
            mx = df[col].max()
            mn = df[col].min()
            if verbose:
                print("min for this col: ",mn)
                print("max for this col: ",mx)
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(df[col]).all(): 
                NAlist.append(col)
                df[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = df[col].fillna(0).astype(np.int64)
            result = (df[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        df[col] = df[col].astype(np.uint8)
                    elif mx < 65535:
                        df[col] = df[col].astype(np.uint16)
                    elif mx < 4294967295:
                        df[col] = df[col].astype(np.uint32)
                    else:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)    
            # Make float datatypes 32 bit
            else:
                df[col] = df[col].astype(np.float32)
            
            # Print new column type
            if verbose:
                print("dtype after: ",df[col].dtype)
                print("******************************")
    # Print final result
    if verbose:
        print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = df.memory_usage().sum() / 1024**2     
    print("Memory usage is: ",mem_usg," MB")    
    print("The dataframe is now ",100*mem_usg/start_mem_usg,"% of the initial size")
    return df, NAlist

**RMSLE calculation** 

In [3]:
def rmsle(y, y_pred):
    '''
    A function to calculate Root Mean Squared Logarithmic Error (RMSLE)
    source: https://www.kaggle.com/marknagelberg/rmsle-function
    '''
    assert len(y) == len(y_pred)
    terms_to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5

In [4]:
# from: https://www.kaggle.com/bejeweled/ashrae-catboost-regressor
def RMSLE(y_true, y_pred, *args, **kwargs):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

**Feature Functions**

In [5]:
def add_lag_feature(weather_df, window=3):
    group_df = weather_df.groupby('site_id')
    cols = ['air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed']
    rolled = group_df[cols].rolling(window=window, min_periods=0)
    lag_mean = rolled.mean().reset_index().astype(np.float16)
    lag_max = rolled.max().reset_index().astype(np.float16)
    lag_min = rolled.min().reset_index().astype(np.float16)
    lag_std = rolled.std().reset_index().astype(np.float16)
    for col in cols:
        weather_df[f'{col}_mean_lag{window}'] = lag_mean[col]
        weather_df[f'{col}_max_lag{window}'] = lag_max[col]
        weather_df[f'{col}_min_lag{window}'] = lag_min[col]
        weather_df[f'{col}_std_lag{window}'] = lag_std[col]

In [6]:
def timestamp_align(df):
    df['offset'] = df.site_id.map(site_ids_offsets)
    df['timestamp_aligned'] = (df.timestamp - pd.to_timedelta(df.offset, unit='H'))
    df['timestamp'] = df['timestamp_aligned']
    del df['timestamp_aligned']
    return df

In [7]:
def mean_without_overflow_fast(col):
    col /= len(col)
    return col.mean() * len(col)

In [8]:
def encode_cyclic_feature(df, col, max_vals):
    df[col + '_sin'] = np.sin(2 * np.pi * df[col]/max_vals)
#     df[col + '_cos'] = np.cos(2 * np.pi * df[col]/max_vals)
    del df[col]
    return df

# Data

In [9]:
building_df = pd.read_csv(PATH+"building_metadata.csv")
weather_train = pd.read_csv(PATH+"weather_train.csv", parse_dates=['timestamp'])
weather_test = pd.read_csv(PATH+"weather_test.csv", parse_dates=['timestamp'])
train = pd.read_csv(PATH+"train.csv", parse_dates=['timestamp'])

## Leak data loading and concat

In [10]:
# load site 0 data
leak_df = pd.read_pickle(PATH+'ucf-spider/site0.pkl') 
leak_df['meter_reading'] = leak_df.meter_reading_scraped
leak_df.drop(['meter_reading_original','meter_reading_scraped'], axis=1, inplace=True)
leak_df.fillna(0, inplace=True)
leak_df.loc[leak_df.meter_reading < 0, 'meter_reading'] = 0
leak_df = leak_df[leak_df.timestamp.dt.year > 2016]
print(len(leak_df))

2260080


In [11]:
del_2016=True
if del_2016:
    bids = leak_df.building_id.unique()
    train = train[train.building_id.isin(bids) == False]

leak_df = leak_df[leak_df.timestamp.dt.year.isin([2017,2018])]

train = pd.concat([train, leak_df])
train.reset_index(inplace=True)
#weather_train = pd.concat([weather_train_df, weather_test_df])
#weather_train_df.reset_index(inplace=True)


### Prepare training and test

In [12]:
weather = pd.concat([weather_train,weather_test],ignore_index=True)
weather_key = ['site_id', 'timestamp']

temp_skeleton = weather[weather_key + ['air_temperature']].drop_duplicates(subset=weather_key).sort_values(by=weather_key).copy()

# calculate ranks of hourly temperatures within date/site_id chunks
temp_skeleton['temp_rank'] = temp_skeleton.groupby(['site_id', temp_skeleton.timestamp.dt.date])['air_temperature'].rank('average')

# create a dataframe of site_ids (0-16) x mean hour rank of temperature within day (0-23)
df_2d = temp_skeleton.groupby(['site_id', temp_skeleton.timestamp.dt.hour])['temp_rank'].mean().unstack(level=1)

# Subtract the columnID of temperature peak by 14, getting the timestamp alignment gap.
site_ids_offsets = pd.Series(df_2d.values.argmax(axis=1) - 14)
site_ids_offsets.index.name = 'site_id'

In [13]:
weather_train = timestamp_align(weather_train)
weather_test = timestamp_align(weather_test)

In [14]:
add_lag_feature(weather_train, window=3)
add_lag_feature(weather_train, window=72)
add_lag_feature(weather_test, window=3)
add_lag_feature(weather_test, window=72)

In [15]:
# Fill NaNs in weather data by interpolation
weather_train = weather_train.groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both'))
weather_test = weather_test.groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both'))

In [16]:
train = train.merge(building_df, left_on = "building_id", right_on = "building_id", how = "left")
train = train.merge(weather_train, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"])

In [17]:
del weather_train
gc.collect()

0

In [18]:
#test = test.merge(weather_test, left_on = ["timestamp"], right_on = ["timestamp"])
#del weather_test

# Simple FE: Timestamp

In [19]:
#train["timestamp"] = pd.to_datetime(train["timestamp"])
train["hour"] = train["timestamp"].dt.hour
train["day"] = train["timestamp"].dt.day
train["weekend"] = train["timestamp"].dt.weekday
train["month"] = train["timestamp"].dt.month
train["year"] = train["timestamp"].dt.year
train['dayofweek'] = train['timestamp'].dt.dayofweek

Removing weired data on site_id 0. This data looks weired until May 20. All electricity meter is 0 until May 20 for site_id == 0. I will remove these data from training data. It corresponds to building_id <= 104

In [20]:
train = train.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

**Delete time stamp and encode ```primary_use```**

In [21]:
train = train.drop("timestamp", axis = 1)
le = LabelEncoder()
train["primary_use"] = le.fit_transform(train["primary_use"])

In [22]:

categoricals = ["building_id", "primary_use", "hour", "day", "weekend", "month", "meter", 'year']

drop_cols = ["precip_depth_1_hr", "sea_level_pressure", "wind_direction", "wind_speed"]

numericals = ["square_feet", "year_built", "air_temperature", "cloud_coverage",
              "dew_temperature"]

weather_lag_cols=['air_temperature_mean_lag3', 'air_temperature_max_lag3',
       'air_temperature_min_lag3', 'air_temperature_std_lag3',
       'cloud_coverage_mean_lag3', 'cloud_coverage_max_lag3',
       'cloud_coverage_min_lag3', 'cloud_coverage_std_lag3',
       'dew_temperature_mean_lag3', 'dew_temperature_max_lag3',
       'dew_temperature_min_lag3', 'dew_temperature_std_lag3',
       'precip_depth_1_hr_mean_lag3', 'precip_depth_1_hr_max_lag3',
       'precip_depth_1_hr_min_lag3', 'precip_depth_1_hr_std_lag3',
       'sea_level_pressure_mean_lag3', 'sea_level_pressure_max_lag3',
       'sea_level_pressure_min_lag3', 'sea_level_pressure_std_lag3',
       'wind_direction_mean_lag3', 'wind_direction_max_lag3',
       'wind_direction_min_lag3', 'wind_direction_std_lag3',
       'wind_speed_mean_lag3', 'wind_speed_max_lag3', 'wind_speed_min_lag3',
       'wind_speed_std_lag3', 'air_temperature_mean_lag72',
       'air_temperature_max_lag72', 'air_temperature_min_lag72',
       'air_temperature_std_lag72', 'cloud_coverage_mean_lag72',
       'cloud_coverage_max_lag72', 'cloud_coverage_min_lag72',
       'cloud_coverage_std_lag72', 'dew_temperature_mean_lag72',
       'dew_temperature_max_lag72', 'dew_temperature_min_lag72',
       'dew_temperature_std_lag72', 'precip_depth_1_hr_mean_lag72',
       'precip_depth_1_hr_max_lag72', 'precip_depth_1_hr_min_lag72',
       'precip_depth_1_hr_std_lag72', 'sea_level_pressure_mean_lag72',
       'sea_level_pressure_max_lag72', 'sea_level_pressure_min_lag72',
       'sea_level_pressure_std_lag72', 'wind_direction_mean_lag72',
       'wind_direction_max_lag72', 'wind_direction_min_lag72',
       'wind_direction_std_lag72', 'wind_speed_mean_lag72',
       'wind_speed_max_lag72', 'wind_speed_min_lag72', 'wind_speed_std_lag72']

feat_cols = categoricals + numericals + weather_lag_cols[:20]

In [23]:
target = np.log1p(train["meter_reading"])

In [24]:
train = train.drop(drop_cols + ["site_id","floor_count","meter_reading"], axis = 1)
#train.fillna(-999, inplace=True)

In [25]:
train, NAlist = reduce_mem_usage(train)

Memory usage is:  4738.387724876404  MB
The dataframe is now  108.75 % of the initial size


## Missing Values - Fill with mean value

In [26]:
missing_values = (100-train.count() / len(train) * 100).sort_values(ascending=False)

In [27]:
missing_values = (100-train.count() / len(train) * 100).sort_values(ascending=False)

In [28]:
missing_features = train.loc[:, missing_values > 0.0]
missing_features = missing_features.apply(mean_without_overflow_fast)

In [29]:
for key in train.loc[:, missing_values > 0.0].keys():
    if key == 'year_built' or key == 'floor_count':
        train[key].fillna(math.floor(missing_features[key]), inplace=True)
        #test[key].fillna(math.floor(missing_features[key]), inplace=True)
    else:
        train[key].fillna(missing_features[key], inplace=True)
        #test[key].fillna(missing_features[key], inplace=True)

## Validation

In [30]:
# target = np.log1p(train["meter_reading"])
# raw_target = np.expm1(target)

In [31]:
num_folds = 5
kf = KFold(n_splits = num_folds, shuffle = True, random_state = 42)
error = 0

for fold, (train_index, val_index) in enumerate(kf.split(train, target)):

    print ('Training FOLD ',fold,'\n')
    print('Train index:','\tfrom:',train_index.min(),'\tto:',train_index.max())
    print('Valid index:','\tfrom:',val_index.min(),'\tto:',val_index.max(),'\n')
    
    train_X = train[feat_cols].iloc[train_index]
    val_X = train[feat_cols].iloc[val_index]
    train_y = target.iloc[train_index]
    val_y = target.iloc[val_index]
    lgb_train = lgb.Dataset(train_X, train_y)
    lgb_eval = lgb.Dataset(val_X, val_y)
    
    params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'rmse'},
            'learning_rate': 0.1,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.9
            }
    
    gbm = lgb.train(params,
                lgb_train,
                num_boost_round=2000,
                valid_sets=(lgb_train, lgb_eval),
               early_stopping_rounds=20,
               verbose_eval = 20)

    y_pred = gbm.predict(val_X, num_iteration=gbm.best_iteration)
    error += np.sqrt(mean_squared_error(y_pred, (val_y)))/num_folds
    
    print('\nFold',fold,' Score: ',np.sqrt(mean_squared_error(y_pred, val_y)))
    #print('RMSLE: ', rmsle(y_pred, val_y))
    #print('RMSLE_2: ', np.sqrt(mean_squared_log_error(y_pred, (val_y))))

    del train_X, val_X, train_y, val_y, lgb_train, lgb_eval
    gc.collect()

    print (20*'---')
    break
    
print('CV error: ',error)


Training FOLD  0 

Train index: 	from: 0 	to: 19036626
Valid index: 	from: 1 	to: 19036625 

Training until validation scores don't improve for 20 rounds.
[20]	training's rmse: 1.55648	valid_1's rmse: 1.55673
[40]	training's rmse: 1.43033	valid_1's rmse: 1.43078
[60]	training's rmse: 1.36326	valid_1's rmse: 1.364
[80]	training's rmse: 1.31716	valid_1's rmse: 1.31772
[100]	training's rmse: 1.28167	valid_1's rmse: 1.28229
[120]	training's rmse: 1.25104	valid_1's rmse: 1.25162
[140]	training's rmse: 1.2252	valid_1's rmse: 1.22582
[160]	training's rmse: 1.19995	valid_1's rmse: 1.20064
[180]	training's rmse: 1.18008	valid_1's rmse: 1.18081
[200]	training's rmse: 1.16378	valid_1's rmse: 1.16455
[220]	training's rmse: 1.14361	valid_1's rmse: 1.14439
[240]	training's rmse: 1.12829	valid_1's rmse: 1.12911
[260]	training's rmse: 1.10962	valid_1's rmse: 1.11048
[280]	training's rmse: 1.09501	valid_1's rmse: 1.09596
[300]	training's rmse: 1.08027	valid_1's rmse: 1.08123
[320]	training's rmse: 1.06

In [32]:
# memory allocation
#del train, target
del target
gc.collect()

0

## Prepare Test

In [33]:
test = pd.read_csv(PATH+"/test.csv", parse_dates=['timestamp'])

In [34]:
test = test.merge(building_df, left_on = "building_id", right_on = "building_id", how = "left")
test = test.merge(weather_test, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"], how='left')

In [35]:
del weather_test
del building_df
gc.collect()

0

In [36]:
test["primary_use"] = le.transform(test["primary_use"])

**Change dates type**

In [37]:
#test["timestamp"] = pd.to_datetime(test["timestamp"])
test["hour"] = test["timestamp"].dt.hour.astype(np.uint8)
test["day"] = test["timestamp"].dt.day.astype(np.uint8)
test["weekend"] = test["timestamp"].dt.weekday.astype(np.uint8)
test["month"] = test["timestamp"].dt.month.astype(np.uint8)
test["year"] = test["timestamp"].dt.year.astype(np.uint8)
test['dayofweek'] = test['timestamp'].dt.dayofweek.astype(np.uint8)

test = test[feat_cols]

In [38]:
for key in train.loc[:, missing_values > 0.0].keys():
    if key == 'year_built' or key == 'floor_count':
        #train[key].fillna(math.floor(missing_features[key]), inplace=True)
        test[key].fillna(math.floor(missing_features[key]), inplace=True)
    else:
        #train[key].fillna(missing_features[key], inplace=True)
        test[key].fillna(missing_features[key], inplace=True)

**Reduce Memory**

In [39]:
test, NAlist = reduce_mem_usage(test)

Memory usage is:  4573.081970214844  MB
The dataframe is now  98.2905982905983 % of the initial size


In [40]:
del weather, weather_key, train_index, val_index
gc.collect()

0

In [41]:
test.head()

,building_id,primary_use,hour,day,weekend,month,meter,year,square_feet,year_built,...,dew_temperature_min_lag3,dew_temperature_std_lag3,precip_depth_1_hr_mean_lag3,precip_depth_1_hr_max_lag3,precip_depth_1_hr_min_lag3,precip_depth_1_hr_std_lag3,sea_level_pressure_mean_lag3,sea_level_pressure_max_lag3,sea_level_pressure_min_lag3,sea_level_pressure_std_lag3
0,0,0,0,1,6,1,0,225,7432,2008,...,12.796875,0.288574,0.0,0.0,0.0,0.0,1022.0,1022.5,1022.0,0.099976
1,1,0,0,1,6,1,0,225,2720,2004,...,12.796875,0.288574,0.0,0.0,0.0,0.0,1022.0,1022.5,1022.0,0.099976
2,2,0,0,1,6,1,0,225,5376,1991,...,12.796875,0.288574,0.0,0.0,0.0,0.0,1022.0,1022.5,1022.0,0.099976
3,3,0,0,1,6,1,0,225,23685,2002,...,12.796875,0.288574,0.0,0.0,0.0,0.0,1022.0,1022.5,1022.0,0.099976
4,4,0,0,1,6,1,0,225,116607,1975,...,12.796875,0.288574,0.0,0.0,0.0,0.0,1022.0,1022.5,1022.0,0.099976


## Inference

In [42]:
from tqdm import tqdm
i=0
res=[]
step_size = 50000 
for j in tqdm(range(int(np.ceil(test.shape[0]/50000)))):
    res.append(np.expm1(gbm.predict(test.iloc[i:i+step_size])))
    i+=step_size

100%|██████████| 834/834 [16:04<00:00,  1.16s/it]


# Submission

In [43]:
res = np.concatenate(res)
sub = pd.read_csv(PATH+"sample_submission.csv")
sub["meter_reading"] = res
sub.to_csv("submission.csv", index = False)
sub.head(10)

,row_id,meter_reading
0,0,8.295210
1,1,4.518826
2,2,8.054509
3,3,55.583306
4,4,557.602135
5,5,8.930230
6,6,68.643231
7,7,350.519425
8,8,110.846266
9,9,186.549073


In [44]:
res.shape

(41697600,)

In [45]:
test.shape

(41697600, 33)

In [46]:
sub.shape

(41697600, 2)

In [47]:
sub.round(1).to_csv("submission_rounded.csv.gz", index = False, compression='gzip')
